# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np

In [2]:
#import tweet_sentiment_extraction data set
# https://huggingface.co/datasets/mteb/tweet_sentiment_extraction
splits = ["train", "test"]

dataset = {split: load_dataset("mteb/tweet_sentiment_extraction", split=split) for split in splits}

#check dataset
print("Dataset: ", dataset["train"])

#inspect dataset
print("Dataset sample:", dataset["train"][0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset:  Dataset({
    features: ['id', 'text', 'label', 'label_text'],
    num_rows: 27481
})
Dataset sample: {'id': 'cb774db0d1', 'text': ' I`d have responded, if I were going', 'label': 1, 'label_text': 'neutral'}


In [3]:
#define model name
model_name = "distilbert-base-uncased"
#load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

#define preprocess function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = dataset[split].map(preprocess_function, batched=True)

#check tockenized ds
print("Tokenized DS: ", tokenized_ds["train"])

# Show the first example of the tokenized training set
print("Tokenized Input ids: ", tokenized_ds["train"][0]["input_ids"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

Tokenized DS:  Dataset({
    features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
    num_rows: 27481
})
Tokenized Input ids:  [101, 1045, 1036, 1040, 2031, 5838, 1010, 2065, 1045, 2020, 2183, 102]


In [4]:
num_labels=3
id2label={0: 'negative', 1: 'neutral', 2: 'positive'}
label2id={'negative': 0, 'neutral': 1, 'positive':2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)
print("Model: ", model)

# Freeze the base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model:  DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=

In [5]:
# train model

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

training_args = TrainingArguments(
    output_dir="./data/ouput/"+model_name,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()
prefine_tuning_results = trainer.evaluate()
print("Evaluation results before fine-tuning: ", prefine_tuning_results)



You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.068400,0.944246,0.564516
2,0.908200,0.857853,0.620260
3,0.858100,0.825050,0.633843
4,0.835200,0.811868,0.641200
5,0.824900,0.807983,0.640917


Evaluation results before fine-tuning:  {'eval_loss': 0.8079832196235657, 'eval_accuracy': 0.6409168081494058, 'eval_runtime': 4.3099, 'eval_samples_per_second': 819.969, 'eval_steps_per_second': 25.755, 'epoch': 5.0}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [24]:
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, AutoPeftModelForSequenceClassification


In [39]:
# LoRA configuration
peft_config = LoraConfig(
    task_type='SEQ_CLS',
    target_modules=["q_lin", "k_lin", "v_lin"], 
    r=8,  
    lora_alpha=32,  
    lora_dropout=0.1 
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

#get peft model
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,406,982 || all params: 67,769,862 || trainable%: 2.076117552076467


In [40]:
#using same params as non-peft, just to see the difference if there is any
peft_training_args = TrainingArguments(
    output_dir="./data/ouput/peft/"+model_name,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True
)

peft_trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

peft_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.017400,0.616839,0.743067
2,0.619700,0.583924,0.761743
3,0.596100,0.570196,0.768251


KeyboardInterrupt: 

In [27]:
#saved pretrained
peft_model.save_pretrained("peft-fine-tuned")
#print model
peft_model


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=Fals

In [28]:
#peft_model.merge_and_unload()

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [36]:
loaded_peft_model = AutoPeftModelForSequenceClassification.from_pretrained("peft-fine-tuned",
                                                                            num_labels=num_labels,
                                                                           id2label=id2label,
                                                                         label2id=label2id,
                                                                         ignore_mismatched_sizes=True)

print("Fine-tuned loaded model: ", loaded_peft_model);

# Set the model's pad token id to match the tokenizer's pad token id
loaded_peft_model.config.pad_token_id = tokenizer.pad_token_id

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-tuned loaded model:  PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768,

In [37]:
#new trainer with the loaded peft model

loaded_training_args = TrainingArguments(
    output_dir="./data/ouput/peft/"+model_name,
    num_train_epochs=1, # one epoch for training
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True
)

loaded_peft_trainer = Trainer(
    model=loaded_peft_model,
    args=loaded_training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

#loaded_peft_trainer.train()

In [38]:
#evaluate the new peft results
loaded_peft_results = loaded_peft_trainer.evaluate()

print("Evaluation results with fine-tuning: ", loaded_peft_results)

Evaluation results with fine-tuning:  {'eval_loss': 1.0234726667404175, 'eval_accuracy': 0.4742501414827391, 'eval_runtime': 4.5202, 'eval_samples_per_second': 781.824, 'eval_steps_per_second': 24.556}


In [33]:
peft_results = peft_trainer.evaluate()

print("Evaluation results with fine-tuning: ", peft_results)

Evaluation results with fine-tuning:  {'eval_loss': 0.5487878918647766, 'eval_accuracy': 0.7730616864742501, 'eval_runtime': 4.5627, 'eval_samples_per_second': 774.543, 'eval_steps_per_second': 24.328, 'epoch': 5.0}
